In [1]:
# INFO: This notebook is for reference purposes only, it describes how ADM.duckdb was populated and may not work in the future (read only DB)

import pandas as pd
import geopandas as gpd

from satellite.weather.orm.functional import session
from satellite.weather.orm.models import ADM0, ADM1, ADM2

### ARG 🇦🇷

#### ADM1

In [2]:
df_adm1 = gpd.read_file("/home/bida/Downloads/provincias.geojson") # https://github.com/PoliticaArgentina/data_warehouse/blob/master/geoAr/data_raw/provincias.geojson
df_adm1 = df_adm1[["codprov_censo", "provincia"]]
df_adm1 = df_adm1.rename(columns={"codprov_censo": "adm1", "provincia": "name"})
df_adm1["adm0"] = "ARG"
df_adm1.head()

,adm1,name,adm0
0,02,Ciudad Autónoma de Buenos Aires,ARG
1,10,Catamarca,ARG
2,14,Córdoba,ARG
3,18,Corrientes,ARG
4,22,Chaco,ARG


In [3]:
with session() as sess:
    for adm in [ADM0, ADM1]:
        adm.drop_table()
        adm.create_table()
        
    sess.execute("INSERT INTO adm0 (code, name) VALUES ('ARG', 'Argentina')")
    sess.execute("INSERT INTO adm1 SELECT * FROM df_adm1")
    sess.commit()

In [4]:
ADM0.get(code="ARG")

Argentina

In [5]:
ADM1.filter()[:5]

[Ciudad Autónoma de Buenos Aires, Catamarca, Córdoba, Corrientes, Chaco]

In [6]:
ADM1.filter(name="Catamarca", adm0="ARG")

[Catamarca]

In [7]:
ADM1.filter(code="10", adm0="ARG")

[Catamarca]

#### ADM2

In [8]:
df_adm2 = gpd.read_file("/home/bida/Downloads/censo_2010.geojson") # https://github.com/PoliticaArgentina/data_warehouse/blob/master/geoAr/data_raw/censos/censo_2010.geojson
df_adm2 = df_adm2[["prov_cod", "link", "depto", "geometry"]]
df_adm2 = df_adm2.rename(columns={"prov_cod": "adm1", "link": "adm2", "depto": "name"})
"""
df_gpkg = df_adm2[["adm1", "adm2", "geometry"]]
df_gpkg.to_file("ARG.gpkg", driver="GPKG")
"""
df_adm2.head()

,adm1,adm2,name,geometry
0,02,02001,Comuna 01,"POLYGON ((-58.36801 -34.57821, -58.36798 -34.5..."
1,02,02002,Comuna 02,"POLYGON ((-58.3944 -34.59968, -58.39471 -34.59..."
2,02,02003,Comuna 03,"POLYGON ((-58.4103 -34.59809, -58.40943 -34.59..."
3,02,02004,Comuna 04,"MULTIPOLYGON (((-58.35987 -34.62558, -58.35978..."
4,02,02005,Comuna 05,"POLYGON ((-58.41566 -34.59786, -58.41463 -34.5..."


In [9]:
df_adm2["adm0"] = "ARG"
df_adm2 = df_adm2[["adm2", "name", "adm0", "adm1"]]
df_adm2.head()

,adm2,name,adm0,adm1
0,02001,Comuna 01,ARG,02
1,02002,Comuna 02,ARG,02
2,02003,Comuna 03,ARG,02
3,02004,Comuna 04,ARG,02
4,02005,Comuna 05,ARG,02


In [10]:
with session() as sess:
    ADM2.drop_table()
    ADM2.create_table()

    sess.execute("INSERT INTO adm2 SELECT * FROM df_adm2")
    sess.commit()

In [11]:
ADM2.filter()[:5]

[Comuna 01, Comuna 02, Comuna 03, Comuna 04, Comuna 05]

In [12]:
ADM2.filter(adm1="02", adm0="ARG")

[Comuna 01,
 Comuna 02,
 Comuna 03,
 Comuna 04,
 Comuna 05,
 Comuna 06,
 Comuna 07,
 Comuna 08,
 Comuna 09,
 Comuna 10,
 Comuna 11,
 Comuna 12,
 Comuna 13,
 Comuna 14,
 Comuna 15]

In [13]:
ADM2.get(code="02001", adm0="ARG")

Comuna 01

In [14]:
ADM2.get(name="Comuna 01", adm0="ARG")

Comuna 01